In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from scipy import interpolate
import scipy.io as sio
from numpy import *

min_max_scaler = preprocessing.MinMaxScaler()

In [2]:
#Import dataset
RUL_F001 = np.loadtxt('./cmapss/RUL_FD001.txt')
train_F001 = np.loadtxt('./cmapss/train_FD001.txt')
test_F001 = np.loadtxt('./cmapss/test_FD001.txt')
train_F001[:, 2:] = min_max_scaler.fit_transform(train_F001[:, 2:])
test_F001[:, 2:] = min_max_scaler.transform(test_F001[:, 2:])
train_01_nor = train_F001
test_01_nor = test_F001
print(train_01_nor.shape)

(20631, 26)


In [3]:
#Delete worthless sensors
train_01_nor = np.delete(train_01_nor, [5, 9, 10, 14, 20, 22, 23], axis=1) 
test_01_nor = np.delete(test_01_nor, [5, 9, 10, 14, 20, 22, 23], axis=1) 

In [4]:
#parameters of data process
RUL_max = 125.0  
window_Size = 40 

trainX = []
trainY = []
trainY_bu = []
testX = []
testY = []
testY_bu = []
testInd = []
testLen = []
testX_all = []
testY_all = []
test_len = []

In [5]:
# 取出1~編號上限的引擎
for i in range(1, int(np.max(train_01_nor[:, 0])) + 1):
    # 用np.where取出一個tuple, array存的是所有編號為i的引擎數據
    ind = np.where(train_01_nor[:, 0] == i)
    # 取出tuple 中的ndarray
    ind = ind[0]
    data_temp = train_01_nor[ind, :]
    for j in range(len(data_temp) - window_Size + 1): 
        trainX.append(data_temp[j:j + window_Size, 2:].tolist())
        # j+windowsize 代表目前time total-current=RUL
        train_RUL = len(data_temp) - window_Size - j
        # 看目前的rul跟max差多少
        train_bu = RUL_max - train_RUL
        if train_RUL > RUL_max:
            train_RUL = RUL_max
            train_bu = 0.0
        trainY.append(train_RUL)
        trainY_bu.append(train_bu)

In [6]:
#Test set sliding time window processing
for i in range(1, int(np.max(test_01_nor[:, 0])) + 1):
    ind = np.where(test_01_nor[:, 0] == i)
    ind = ind[0]
    # 編號i的engine的total time
    testLen.append(float(len(ind)))
    data_temp = test_01_nor[ind, :]
    # 編號i的最後time step
    testY_bu.append(data_temp[-1, 1])
    if len(data_temp) < window_Size:
        
        data_temp_a = []
        # 特徵維度
        for myi in range(data_temp.shape[1]):
            # 根據 x數量 產生 0 ~ 40之間的x座標 
            x1 = np.linspace(0, window_Size - 1, len(data_temp))
            # 產生 0~40x座標
            x_new = np.linspace(0, window_Size - 1, window_Size)
            # 根據維度myi的點產生逼近函數
            tck = interpolate.splrep(x1, data_temp[:, myi])
            # 將0~40帶入tck產生的函數得y點
            a = interpolate.splev(x_new, tck)
            data_temp_a.append(a.tolist())
        data_temp_a = np.array(data_temp_a)
        data_temp = data_temp_a.T
        data_temp = data_temp[:, 2:]
    else:
        # test 只要最後四十個預測還有多少RUL即可
        data_temp = data_temp[-window_Size:, 2:]  
    # for concate usage extand 0th dimension
    data_temp = np.reshape(data_temp, (1, data_temp.shape[0], data_temp.shape[1])) 
    testX = data_temp if i == 1 else np.concatenate((testX, data_temp), axis=0)
    if RUL_F001[i - 1] > RUL_max:
        testY.append(RUL_max)
        #testY_bu.append(0.0)
    else:
        testY.append(RUL_F001[i - 1])

In [7]:
trainX = np.array(trainX)
testX = np.array(testX)
trainY = np.array(trainY)/RUL_max 
trainY_bu = np.array(trainY_bu)/RUL_max
testY = np.array(testY)/RUL_max
testY_bu = np.array(testY_bu)/RUL_max

In [8]:
print(trainX.shape)
print(trainY.shape)

(16731, 40, 17)
(16731,)


In [21]:
print(trainX.shape)
print(testX.shape)
print(trainY.shape)
print(testY.shape)

(16731, 40, 17)
(100, 40, 17)
(16731,)
(100,)


In [22]:
sio.savemat('F001_window_size_trainX.mat', {"train1X": trainX})
sio.savemat('F001_window_size_trainY.mat', {"train1Y": trainY})
sio.savemat('F001_window_size_testX.mat', {"test1X": testX})
sio.savemat('F001_window_size_testY.mat', {"test1Y": testY})

In [ ]:
import scipy.io

mat = scipy.io.loadmat('F001_window_size_trainX.mat')
struct_array = mat['']
value = struct_array[0][0]['field_name']